# Creating fibers using 3D coordinates

Note: this tutorial will use a lot of code explained in the [finite amplitudes tutorial](1_finite_amp.ipynb) and the [resampling potentials tutorial](4_resampling_potentials.ipynb), so it is recommended to review them before proceeding.

## Creating 3D fiber path and potentials
Normally, you might use 3D fiber coordinates alongside potentials that you obtained from an external source (e.g., COMSOL, ANSYS, etc.). For the purpose of this tutorial, we will create a fiber path using a 3D spiral and create potentials using a gaussian curve.

In [ ]:
# Creating the spiral path for the fiber
import numpy as np
import matplotlib.pyplot as plt

# Define parameters for the spiral
height = 1000  # total height of the spiral
turns = 5  # number of turns in the spiral
radius = 1000  # radius of the spiral
points_per_turn = 20  # points per turn

# Generate the spiral coordinates
t = np.linspace(0, 2 * np.pi * turns, points_per_turn * turns)
x = radius * np.cos(t)
y = radius * np.sin(t)
z = np.linspace(0, height, points_per_turn * turns)

# Combine into a single array for path coordinates
path_coordinates = np.column_stack((x, y, z))

# Plot the generated path to visualize it
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z)
ax.set_title("3D Spiral Path for Fiber")
ax.set_xlabel("X axis")
ax.set_ylabel("Y axis")
ax.set_zlabel("Z axis")
plt.show()

In [ ]:
# Creating potentials along the fiber
# Calculate the length along the fiber
fiber_length = np.sqrt(np.sum(np.diff(path_coordinates, axis=0) ** 2, axis=1)).sum()

# Generate the Gaussian curve of potentials
# Set the mean at the center of the fiber and standard deviation as a fraction of the fiber length
mean = fiber_length / 2
std_dev = fiber_length / 50  # Smaller value gives a narrower peak

# Create an array representing the linear distance along the fiber
linear_distance = np.linspace(0, fiber_length, len(path_coordinates))

# Generate Gaussian distributed potentials
potentials = np.exp(-((linear_distance - mean) ** 2 / (2 * std_dev**2)))

# Normalize potentials for better visualization or use
potentials /= np.max(potentials)

# set to max=500 mV
potentials *= 500

# Plotting the Gaussian distribution of potentials along the fiber
plt.figure()
plt.plot(linear_distance, potentials, label='Gaussian Potentials')
plt.xlabel('Distance along fiber (μm)')
plt.ylabel('Potential (mV)')
plt.title('Gaussian Distribution of Potentials along fiber path')
plt.legend()
plt.show()

## Resampling 3D potentials
For this tutorial, we will create a fiber model using the MRG model using the same process as the [finite amplitudes tutorial](1_finite_amp.ipynb). Instead of specifying the length or number of sections, we provide the coordinates of the 3D fiber path. Since we want a 3d_fiber, we must use the function `build_fiber_3d()` instead.

In [ ]:
from pyfibers import FiberModel, build_fiber_3d

fiber = build_fiber_3d(FiberModel.MRG_INTERPOLATION, diameter=10, path_coordinates=path_coordinates)
print(fiber)
print('Fiber is 3D?', fiber.is_3d())

We can either:
- use the `resample_potentials_3d()` method of the fiber object to get the potentials at the center of each fiber section. (not recommended)
- calculate arc lengths and use `resample_potentials()` to get the potentials. (recommended)

In [ ]:
# using the resample_potentials_3d method to resample the potentials along the fiber path
fiber.resample_potentials_3d(potentials=potentials, potential_coords=path_coordinates, center=True, inplace=True)

plt.figure()
plt.plot(fiber.longitudinal_coordinates, fiber.potentials, marker='o', label='resample_potentials_3d()')
plt.xlabel('Distance along fiber (μm)')
plt.ylabel('Potential (mV)')
plt.title('Resampled Potentials along Fiber')

# calculating arc lengths and using resample_potentials()
arc_lengths = np.concatenate(([0], np.cumsum(np.sqrt(np.sum(np.diff(path_coordinates, axis=0) ** 2, axis=1)))))
fiber.resample_potentials(potentials=potentials, potential_coords=arc_lengths, center=True, inplace=True)
plt.plot(
    fiber.longitudinal_coordinates, fiber.potentials, marker='x', label='resample_potentials()', alpha=0.6, color='k'
)

## Simulation
We will use the same simulation setup as before.

In [ ]:
import numpy as np
from pyfibers import ScaledStim

# Setup for simulation
waveform = np.concatenate((np.ones(100), -np.ones(100), np.zeros(49800)))  # biphasic rectangular pulse

time_step = 0.001  # milliseconds
time_stop = 15  # milliseconds

# Create stimulation object
stimulation = ScaledStim(waveform=waveform, dt=time_step, tstop=time_stop)

and then run a fixed amplitude simulation:

In [ ]:
ap, time = stimulation.run_sim(-1.5, fiber)
print(f'Number of action potentials detected: {ap}')
print(f'Time of last action potential detection: {time} ms')

## Creating potentials from a point source
In the previous example, we created potentials using a gaussian curve. In this example, we will create potentials using a point source. We will use the same fiber path as before, but we will calculate the potentials from a point source near the 3D fiber path.

In [ ]:
# Calculate point source potentials at all fiber coordinates
x, y, z = 800, 800, 500  # Point source location
i0 = 1  # Current of the point source
sigma = 1  # Isotropic conductivity
fiber_potentials = fiber.point_source_potentials(x, y, z, i0, sigma, inplace=True)

# plot
plt.figure()
plt.plot(fiber.longitudinal_coordinates, fiber.potentials, marker='o')
plt.xlabel('Distance along fiber (μm)')
plt.ylabel('Potential (mV)')
plt.title('Fiber potentials from point source')
plt.show()

The potentials have several peaks, which would be expected from the location where we placed the point source.

In [ ]:
# plot the fiber with the point source
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, 'ro', label='Point Source')
ax.plot(fiber.coordinates[:, 0], fiber.coordinates[:, 1], fiber.coordinates[:, 2], label='Fiber Path')
ax.set_title('Fiber Path with Point Source')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.legend()